In [1]:
import pandas as pd    
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [2]:
# Launch web driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======


2022-10-27 04:16:08,781 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 107.0.5304


2022-10-27 04:16:08,886 INFO Current google-chrome version is 107.0.5304


[WDM] - Get LATEST chromedriver version for 107.0.5304 google-chrome


2022-10-27 04:16:08,887 INFO Get LATEST chromedriver version for 107.0.5304 google-chrome


[WDM] - Driver [/Users/rohitvenkat/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver] found in cache


2022-10-27 04:16:08,991 INFO Driver [/Users/rohitvenkat/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver] found in cache


In [3]:
# Load google maps url
url = 'https://www.google.com/maps/search/?api=1&query=Google&query_place_id=ChIJIS56-XFmZIgRsvTzijiFReo'
driver.get(url)

# Click reviews link
driver.find_element('xpath', '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/button').click()
time.sleep(5)

# Click sort button
wait = WebDriverWait(driver, 10)
menu_bt = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@data-value=\'Sort\']')))  
menu_bt.click()

# Sort by newest reviews
recent_rating_bt = driver.find_elements('xpath', '//li[@data-index=\'1\']')[0]
recent_rating_bt.click()
time.sleep(5)

In [4]:
# Parse html page
response = BeautifulSoup(driver.page_source, 'html.parser')

# Find total number of reviews
total_reviews = int(response.find('div', class_ = 'fontBodySmall').text.replace(' reviews', ''))

# Find scroll layout
scrollable_div = driver.find_element('xpath', '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')

# Scroll as many times as necessary to load all reviews
for i in range(0, (round(total_reviews / 5 - 1))):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(1)

In [5]:
# Re-parse html page after loading all reviews
response = BeautifulSoup(driver.page_source, 'html.parser')

In [6]:
# Extract reviews
review_ids = [review_id['data-review-id'] for review_id in response.findAll('div', class_= 'jftiEf')]
review_dates = [review_date.text for review_date in response.findAll('span', class_ = 'rsqaWe')]
reviewers = [reviewer['aria-label'] for reviewer in response.findAll('div', class_= 'jftiEf')]
ratings = [rating['aria-label'].strip() for rating in response.findAll('span', class_ = 'kvMYJc')]
review_texts = [review_text.text for review_text in response.findAll('span', class_ = 'wiI7pd')]

In [7]:
# Create reviews df
reviews = pd.DataFrame(
    data = {
        'review_id': review_ids,
        'date': review_dates,
        'reviewer': reviewers,
        'rating': ratings,
        'text': review_texts
    }
)

In [8]:
reviews.head()

,review_id,date,reviewer,rating,text
0,ChZDSUhNMG9nS0VJQ0FnSURlN19fR1RBEAE,4 days ago,Ana,5 stars,"This is a great place, great drinks, great mus..."
1,ChdDSUhNMG9nS0VJQ0FnSURlbVltV2xnRRAB,a week ago,Pemberton Heath,5 stars,"Awesome bar, incredible restaurant"
2,ChRDSUhNMG9nS0VJQ0FnSURlaFBGchAB,3 weeks ago,Shelley Ingham,5 stars,The nachos are amazing
3,ChZDSUhNMG9nS0VJQ0FnSUNleW82eGFBEAE,a month ago,Amy Garten,5 stars,
4,ChdDSUhNMG9nS0VJQ0FnSUNlaU1pejFBRRAB,a month ago,Alfredo arguello,5 stars,Bastion is an outstanding culinary experience ...


TODO:
- Extract reviewer ID
- Extract exact date
- Extract full text